# XGBoost Modeling

In this Notebook XGBoost can be used to make some magic!


In this Notebook XGBoost can be used to make some magic!
Because each of the model needs to be safed properly. Please specify a name for the model, which describes briefly its specialities:

In [ ]:
model_name: str= "Linear_base_XGBoost_Model"

# 1. Setting up Colab

Here we set up colab. U knooow it :D 

In case u don't use colab, please specify the variable ```use_colab``` to false. In case u don't use it its unavoidable to make sure that noone changed something online while u change it offline. The results of this would be worse than a division by zero.  

In [1]:
use_colab: bool = True

In case u accidentally run the following code twice u will get following weird and confusing error:

```
OSError: [Errno 107] Transport endpoint is not connected
```

In this case just restart the runtime above. 


In [2]:
if use_colab:
  import os
  from google.colab import drive 
  drive.mount("/content/gdrive", force_remount=True)
  # Change directory for nicer imports
  %cd "/content/gdrive/My Drive/Data_Mining_Cup/05 Code/"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Data_Mining_Cup/05 Code


# 2. Nodes and Ideas

## Ideas for new Features
Please specify assignment in bold and brackets behind. If nobody is assigned put the "To Be Assigned (TBA)" Flague.
*

## Notes

Currently no notes :( 

# 3. Setting up Notebook

Here we setup some parameters for the notebook to work in the expected ways.

If we want to **use the final test phase** for the later submission, we need to specify the following parameter **```make_final_submission``` to true.** Otherwise the selfmade test set will be used.

In [3]:
make_final_submission: bool = False

In case we don't want to make a final submission, we now have to specify the test_set. In case you change it, please leave a comment on why and what the new one is. If the end is defined as None we use all the data from start_period (inclusive) till the end of dataset. 

In case make_final_submission is True we do not use those parameters.

In [24]:
# This will set the start of test period to 2 weeks before end of data

test_period_start: str="2018-06-02"
# Use till the end
test_period_end: str="2018-06-15"

In [25]:
# These datapoints are for the final submision
final_submission_start: str="2018-06-30"
final_submission_end: str="2018-07-14"

#### Imports

Here you can add imports u need. Please use alphabetical order to not import new stuff twice.

In [26]:

from datetime import date, datetime, timedelta
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from itertools import product
import numpy as np
from math import sqrt 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
from typing import List, Union

import xgboost as xgb 
from xgboost import plot_importance 

%matplotlib inline

In case of a "Model not found" Error concerning the Hyperopt package. Please uncomment and run the following line

In [27]:
# !pip install hyperopt

## Loading data

Here we now load the dataframes by the preceeding notebooks.
Please do not import single datasets and merge them here cause it will end in fatal inconsistencies.

For comfort the following row will define a ```small_data``` attribute. is this True, only a part will be used for training, if False all the data points will be used in the following code.

It will leave the testing and validation data in the same size, only will decrease the data for training.


In [28]:
small_data: bool = True

In case small data should be used, the following line defines the start of this. 

In [29]:
small_data_start_date: str = '20180418'

For the training of XGBoost, we need to specify a validation set. Herefore we no can set the traning_end. Between this and the start of testing will be used as validation set. 


In [30]:
training_end: str = '20180501'

### Label

The last thing which need to be specified is, which label should be used for the training, so what do u want to be predicted. Sadly because of xg boos we only can use exactly one label :( 

In [31]:
labels: List[str] = ["order"]

Therefore this assert. Remove it if we change to another model

In [32]:
assert len(labels) == 1

## Features to delete for training

Turns out another parameter needs to be set. Which feature do we have in the data and should be deleted for training. The date will be deleted seperately

In [33]:
columns_to_delete: List[str] = []

# 4. Preparation
Here the data is finally fetched

In [34]:
# OPEN PANDAS DATAFRAMES FROM PREPROCESSING 2
try:
  if make_final_submission:
    dynamic_features = pd.read_pickle('./data/dynamic_features_final_nb2.pk')
    static_features = pd.read_pickle('./data/static_features_final_nb2.pk')
  else:
    dynamic_features = pd.read_pickle('./data/dynamic_features_nb2.pk')
    static_features = pd.read_pickle('./data/static_features_nb2.pk')
    labels_for_test = pd.read_pickle('./data/orders_for_test.pkl')
except FileNotFoundError as e:
  print(f"One File could not be found. Please rerun the first Notebook, {e}")

Combine the two datasets together

In [35]:
data = dynamic_features.join(static_features.drop('avg_salesPrice_nonunique', axis=1), on="itemID")
data = data.reset_index()

Here we make the dataframe a bit smaller:

In [36]:
if small_data: 
  data = data[data['date'] > small_data_start_date]

#### Deletion of unused feature


In [37]:
try:
  # do the deletion
  data = data.drop(columns_to_delete, axis=1)
except KeyError as e:
  print(f"I could not delete the columns {e} cause they were not there. \n But hey, they are gone, like intented :D")

#### Adding the label

After this step, data does not exist, only X, y left

In [38]:
# We need to put date also into y to make the split
y = data[['date', *labels]].copy()
X = data.drop(labels, axis=1).copy()

#### Split into train/val/test

Here we create tuples for the respective datapoints.
(X,y) 
In the end we will have train, val and test according to the variables defined above

In [39]:
def split(X, y, training_end, testing_start, split_feature='date', labels_for_test=None, delete_split_feature=True):
  
  # Training
  print(f"CREATING TRAINING from [begin, {training_end}]")
  train_X = X[X[split_feature] <= training_end]
  train_y = y[y[split_feature] <= training_end]

  # Validation
  print(f"CREATING VALIDATION from ]{training_end}, {testing_start}[")
  val_X = X[(X[split_feature] > training_end)]
  val_X = val_X[(val_X[split_feature] < testing_start)]
  val_y = y[(y[split_feature]> training_end)]
  val_y = val_y[(val_y[split_feature] < testing_start)]

  assert len(val_X)>0, "val_X must be greater 0"
  assert len(val_y)>0, "val_y must be greater 0"

   # Testing
  print(f"CREATING TESTING from ({testing_start}, end]")
  test_X = X[X[split_feature] >= testing_start]
  test_y = y[y[split_feature] >= testing_start]
    
  assert len(test_X)>0, "test_X must be greater 0"
  assert len(test_y)>0, "test_y must be greater 0"
    
  if delete_split_feature:
    train_X = train_X.drop([split_feature,'itemID'], axis=1)
    train_y = train_y.drop(split_feature, axis=1)
    val_X = val_X.drop([split_feature,'itemID'], axis=1)
    val_y = val_y.drop(split_feature, axis=1)
    test_X = test_X.drop(split_feature, axis=1)
    test_y = test_y.drop(split_feature, axis=1)
    
  if labels_for_test is not None:
    test_y['order'] = labels_for_test.reset_index()['order'].values
  return (train_X, train_y), (val_X, val_y), (test_X, test_y)

test_start = final_submission_start if make_final_submission else test_period_start
train, val, test = split(X,y, training_end, test_start, labels_for_test=labels_for_test)
del X
del y
#del data

CREATING TRAINING from [begin, 20180501]
CREATING VALIDATION from ]20180501, 2018-06-02[
CREATING TESTING from (2018-06-02, end]


### Save Test, train and val data

Here we safe the preprocessing data.

First the Test data:

In [40]:
import pickle
with open(f"./data/{model_name}_test.pk","wb") as f:
    pickle.dump(test, f)
    f.close()

NameError: ignored

Then the training data

In [41]:
with open(f"./data/{model_name}_train.pk","wb") as f:
    pickle.dump(train, f)
    f.close()

NameError: ignored

At last the validation data

In [42]:
with open(f"./data/{model_name}_val.pk","wb") as f:
    pickle.dump(val, f)
    f.close()

NameError: ignored

# The Model

### Load the data
Here we load the data from above step. __Why?__ This is to make sure that also one of the three main parts of the notebook can be run by themself :) In case all of the notebook is run, this part can be ignored. 

In [43]:
with open(f"./data/{model_name}_train.pk","rb") as f:
    train = pickle.load(f)
    f.close()

with open(f"./data/{model_name}_test.pk","rb") as f:
    test = pickle.load(f)
    f.close()
    
with open(f"./data/{model_name}_val.pk","rb") as f:
    val = pickle.load(f)
    f.close()   

NameError: ignored

## The Custom Evaluation and Objective Function

Here is space to create several custom evaluation and objective functions to later use in XGBoost.

### Karins First approach


In [44]:
# what I'm writing here is very wrong at the moment... (still using rmse atm for training the models)
# how should we get item_price?? -> need itemID, however we don't include itemID for training 

from sklearn.metrics import make_scorer

def custom_loss(y_true, y_pred, item_price):
    if y_true == y_pred:
        revenue = item_price * y_pred 
    elif y_true > y_pred:
        revenue = item_price * y_pred 
    else:
        revenue = item_price * (y_true + 0.6 * (y_true - y_pred)) # here (y_true - y_pred) is negative 
    return revenue 

my_custom_score = make_scorer(custom_loss, greater_is_better=True)

## XGBoost Regressor Model

In case we just need to fast train the model to see if it works we can set this following amount variable. This will set the training data to this amount. Set it to ```None``` if all the data should be used.

In [45]:
amount: Union[int, None] = 1000

### Hyper parameters tuning 

more parameters can be added here in the parameter space: 
range can also be changed for more fine tuning 

In [46]:

params={'booster':'gblinear','objective':'reg:linear'}

The following parameter explains how many rounds for hyperparameter tuning should be used:

In [47]:
evaluation_round: int = 1

### Actual Hyperparameter training
This method returns a evaluation function so it creates a function which creates a model

The actual hyperparameter tuning will be done now:

In [48]:
# set runtime environment to GPU at: Runtime -> Change runtime type 
trials, best_hyperparams = hyperpara_tuning(para_space)
final_xgb_model = trials.best_trial['result']['model']
assert final_xgb_model is not None, "Oooops there is no model created :O "

print("\n\n\n############################################\n")
print("THE BEST PARAMETER WHERE:")
print(best_hyperparams)

NameError: ignored

**Fit Model**

In [49]:
def linear_xgb_model(x_train, y_train, x_val, y_val):
  train = xgb.DMatrix(data=x_train, label=y_train)
  
  xg_reg = xgb.train(dtrain=train,  params=params, num_boost_round=200)

    
  return xg_reg

In [50]:
lin_xgb_model = linear_xgb_model(train[0], train[1], val[0], val[1])

[14:26:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [51]:
dtest = xgb.DMatrix(data=val[0], label=val[1])
dtrain = xgb.DMatrix(data=train[0], label=train[1])


In [ ]:

train_preds = lin_xgb_model.predict(dtrain)


In [ ]:
train_preds = lin_xgb_model.predict(dtrain)
train_preds = [round(i) for i in train_preds]
print("Train RMSE : %.8g" % sqrt(metrics.mean_squared_error(train[1], train_preds)))

val_preds = lin_xgb_model.predict(dtest)
val_preds = [round(i) for i in val_preds]
print("Val RMSE : %.8g" % sqrt(metrics.mean_squared_error(val[1], val_preds)))

In [ ]:
# Compute and print the RMSE
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(train[1], train_preds))
print("RMSE: %f" % (rmse))

In [ ]:
ax = plot_importance(lin_xgb_model)
fig = ax.figure
fig.set_size_inches(54, 42)

### Save the model

Here we now save the model. To distinguish between used models, we can use suffixes. A suffix will then be used to find the right model. If the following variable equals None, simply the current date will be taken as suffix. The final file then is called {modelname}_{suffix}.

In [ ]:
suffix: str= None

---

In [ ]:
suffix = suffix if suffix else datetime.now().strftime('%m%d%Y')
with open(f"./models/{model_name}_{suffix}.pk","wb") as f:
    pickle.dump(final_xgb_model, f)
    f.close()

# Prediction

This part is used to save the model, and create the predictions already. In case we run the whole notebook, no changes need to be made. 
In case **only this part** should be executed, we need to input the name of the model.  Because we know already the name of the model (see upthere somewhere) we only need the suffix (Most probably the date of training).


In [ ]:
suffix_for_import = None

### Load Model

In [ ]:
if final_xgb_model is None:
    suffix_for_import = suffix_for_import if suffix_for_import else datetime.now().strftime('%m%d%Y')
    with open(f"./models/{model_name}_{suffix_for_import}.pk","rb") as f:
        final_xgb_model = pickle.load(f)
        f.close()
    print("Loaded model")
else:
    print("Use the already created model")

NameError: ignored

In [ ]:
with open(f"./data/{model_name}_test.pk","rb") as f:
    test = pickle.load(f)
    f.close()

FileNotFoundError: ignored

Here we also load the data again

In [ ]:
with open(f"./data/{model_name}_train.pk","rb") as f:
    train = pickle.load(f)
    f.close()

with open(f"./data/{model_name}_test.pk","rb") as f:
    test = pickle.load(f)
    f.close()
    
with open(f"./data/{model_name}_val.pk","rb") as f:
    val = pickle.load(f)
    f.close()   

FileNotFoundError: ignored

## Recursive Prediction
**STILL IN WORK**

When doing recursive predictions, we predict the next value and feed them into the data for the next prediction. The following code line enable a recursive prediction.
The following line specifies the feature which should be inserted in each iteration, in 99% of times this will be orders.

In [ ]:
sliding_feature_name:str = "order"

In the next step we need to get the derived lack features. (They were created in notebook no. 2 therefore will have a ** _t-x** suffix.)

In [ ]:
# Maximum lack size
_max_wind_size = 100
# All possible columns
_possible_col = set([f"{sliding_feature_name}_t-{i}" for i in range(_max_wind_size)])
sliding_features = _col.intersection(set(train[0].columns))
_idx = list(map(lambda x: int(x.split('order_t-')[1]), sliding_features))
_idx.sort()

NameError: ignored

In the next step we need to get the derived lack features. (They were created in notebook no. 2 therefore will have a **_t-x** suffix.)

In [ ]:
# Maximum lack size
_max_wind_size = 100
# All possible columns
_possible_col = set([f"{sliding_feature_name}_t-{i}" for i in range(_max_wind_size)])
sliding_features = _col.intersection(set(train[0].columns))
_idx = list(map(lambda x: int(x.split('order_t-')[1]), sliding_features))
_idx.sort()

# Create a mapping for later renaming
mapping = {f"{sliding_feature_name}_t-{max(_idx)}":f"{sliding_feature_name}_t-0" }
for i in range(max(_idx)-1):
    mapping[f"{sliding_feature_name}_t-{i}"] = f"{sliding_feature_name}_t-{i+1}"

# first prediction
pred = final_xgb_model.predict(test[0].iloc[[1]].drop('itemID', axis=1))

for itemID in set(list(test[0]['itemID'])):
    _test_data = test[0].query('itemID==1')
    

In [ ]:
_test_data = copy.deepcopy(test[0].query('itemID==1'))

_pred_row = _test_data.iloc[[0]].copy()
_new_row = _test_data.iloc[[1]].copy()
_pred = final_xgb_model.predict(_test_data.iloc[[0]].drop('itemID', axis=1))
_new_row.iloc[0, _new_row.columns.get_loc(f"{sliding_feature_name}_t-1")] = pred_1[0]  
_pred2 = final_xgb_model.predict(_test_data.iloc[[0]].drop('itemID', axis=1))
_pred_row = _new_row.copy()
_new_row = _new_row.rename(mapping)
_new_row.rename(mapping).T
    #pred = final_xgb_model.predict(_test_data.loc[[idx]].drop('itemID', axis=1))

# set value in next row 
test[0].iloc[1,test[0].columns.get_loc(f"{sliding_feature_name}_t-1")] = pred_1[0]

pred_2 = final_xgb_model.predict(test[0].iloc[[1]])

next_test = test[0].iloc[[1]].rename(mapping)

pred_3 = final_xgb_model.predict(next_test)

next_test.iloc[0].rename(mapping)